#自定义图层

我们建议将其tf.keras用作构建神经网络的高级API。也就是说，大多数TensorFlow API都可用于急切执行。

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

tf.enable_eager_execution()

##图层：常用的有用操作集
大多数情况下，为机器学习模型编写代码时，您希望在比单个操作和单个变量操作更高的抽象级别上操作。

许多机器学习模型都可以表达为相对简单的层的组合和堆叠，而TensorFlow提供了一组许多常用层，以及您从头开始或作为组合创建自己的应用程序特定层的简单方法。现有的图层。

TensorFlow包括完整Keras在tf.keras封装API，并建立自己的模型时，Keras层是非常有用的。

In [0]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
#simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to 
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

可以在[文档中](https://https://www.tensorflow.org/api_docs/python/tf/keras/layers)看到预先存在的图层的完整列表。它包括Dense（完全连接层），Conv2D，LSTM，BatchNormalization，Dropout等等。

In [3]:
# To use a layer, simply call it.
layer(tf.zeros([10,5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [4]:
# Layers have many useful methods. For example, you can inspect all variables
# in a layer using `layer.variables` and trainable variables using
# `layer.trainable_variables`. In this case a fully-connected layer
# will have variables for weights and biases.
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.15839541,  0.33652818,  0.05446959,  0.2981816 ,  0.34496832,
          0.34179062,  0.0978961 , -0.2944942 , -0.49470466,  0.58395964],
        [ 0.09877837, -0.5166745 , -0.22292516, -0.44088125,  0.50255007,
          0.06522959, -0.6230375 ,  0.56266683,  0.24604386, -0.27850106],
        [-0.51025283, -0.19881204, -0.0418992 ,  0.29163212, -0.11847174,
         -0.39820457, -0.18268648,  0.15275526,  0.07371175, -0.5352407 ],
        [ 0.48866326,  0.25386983,  0.0460397 , -0.5651083 , -0.3633575 ,
          0.13588065, -0.2829908 , -0.21680239, -0.579241  ,  0.595136  ],
        [-0.17696479, -0.3160016 , -0.280664  , -0.33502635,  0.03826153,
          0.6114345 , -0.52762026,  0.03110737, -0.49838755, -0.403283  ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [5]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.15839541,  0.33652818,  0.05446959,  0.2981816 ,  0.34496832,
          0.34179062,  0.0978961 , -0.2944942 , -0.49470466,  0.58395964],
        [ 0.09877837, -0.5166745 , -0.22292516, -0.44088125,  0.50255007,
          0.06522959, -0.6230375 ,  0.56266683,  0.24604386, -0.27850106],
        [-0.51025283, -0.19881204, -0.0418992 ,  0.29163212, -0.11847174,
         -0.39820457, -0.18268648,  0.15275526,  0.07371175, -0.5352407 ],
        [ 0.48866326,  0.25386983,  0.0460397 , -0.5651083 , -0.3633575 ,
          0.13588065, -0.2829908 , -0.21680239, -0.579241  ,  0.595136  ],
        [-0.17696479, -0.3160016 , -0.280664  , -0.33502635,  0.03826153,
          0.6114345 , -0.52762026,  0.03110737, -0.49838755, -0.403283  ]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

##实现自定义图层
实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：* __init__，您可以在其中执行所有与输入无关的初始化* build，您可以在其中了解输入张量的形状并可以执行其余的初始化* call，在哪里进行正向计算

请注意，您不必等到build调用创建变量，您也可以在其中创建变量__init__。但是，创建它们的优点build是它可以根据图层将要操作的输入的形状启用后期变量创建。另一方面，创建变量__init__意味着需要明确指定创建变量所需的形状。

In [6]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs
    
  def build(self, input_shape):
    self.kernel = self.add_variable("kernel",
                                   shape=[int(input_shape[-1]),
                                         self.num_outputs])
    
  def call(self, input):
    return tf.matmul(input, self.kernel)
  
layer = MyDenseLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[ 4.95858967e-01,  2.86221325e-01, -2.06197232e-01,
         4.81222928e-01, -5.64075828e-01, -4.57111895e-01,
        -5.44994473e-02, -2.85998166e-01,  4.69181955e-01,
         4.96079981e-01],
       [ 2.38072395e-01, -5.10455012e-01, -1.04665756e-04,
         4.84037399e-02, -2.16632903e-01, -4.34904695e-02,
         3.53232324e-01,  5.11702478e-01,  5.98318040e-01,
        -4.22724545e-01],
       [-3.92193675e-01, -3.97662520e-01,  4.28095400e-01,
         1.44044340e-01, -2.72432566e-01,  3.95300806e-01,
        -4.60255682e-01, -

如果尽可能使用标准层，则整体代码更易于阅读和维护，因为其他读者将熟悉标准层的行为。如果你想使用tf.keras.layers或tf.contrib.layers中不存在的图层，请考虑提交github问题，或者更好的是向我们发送拉取请求！

##模型：组成图层
机器学习模型中许多有趣的层状事物是通过组合现有层来实现的。例如，resnet中的每个残余块是卷积，批量标准化和快捷方式的组合。

创建包含其他图层的类似图层的东西时使用的主类是tf.keras.Model。实现一个是通过继承自tf.keras.Model完成的。

In [7]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters
    
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()
    
    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()
    
    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()
    
  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)
    
    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)
    
    x = self.conv2c(x)
    x = self.bn2c(x, training=training)
    
    x += input_tensor
    return tf.nn.relu(x)
  
block = ResnetIdentityBlock(1, [1,2,3])
print(block(tf.zeros([1,2,3,3])))
print([x.name for x in block.trainable_variables])

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization/gamma:0', 'resnet_identity_block/batch_normalization/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_1/gamma:0', 'resnet_identity_block/batch_normalization_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_2/gamma:0', 'resnet_identity_block/batch_normalization_2/beta:0']


然而，在很多时候，组成许多层的模型只是将一层接一层地称为一层。这可以使用`tf.keras.Sequential`在非常少的代码中完成

In [8]:
my_seq = tf.keras.Sequential([
    tf.keras.layers.Conv2D(1,(1,1)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(2,1,padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (1,1)),
    tf.keras.layers.BatchNormalization()
])

my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=514, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

##下一步
现在，您可以返回到之前的笔记本并调整线性回归示例，以使用更好的结构化图层和模型。